<a href="https://colab.research.google.com/github/RyuMinHo/GAI_project/blob/main/image_gradio_gui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install gradio vllm transformers triton PyMuPDF Pillow sentence_transformers numpy typing open_clip_torch faiss-gpu
!python -m spacy download en_core_web_sm

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 5

In [2]:

import torch
from vllm import LLM
import spacy
from sentence_transformers import SentenceTransformer
from vllm.sampling_params import SamplingParams
import open_clip
from PIL import Image
import torch.nn.functional as F

def init_models():
    try:
        print("LLaVA 모델 로딩 중...")
        llm = LLM(model="llava-hf/llava-v1.6-mistral-7b-hf",
                  dtype='half',
                  max_model_len=8192,
                  device="cuda" if torch.cuda.is_available() else "cpu")

        print("CLIP 모델 로딩 중...")
        model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
        tokenizer = open_clip.get_tokenizer('ViT-B-32')

        print("SpaCy 모델 로딩 중...")
        nlp = spacy.load("en_core_web_sm")

        print("SentenceTransformer 로딩 중...")
        embedder = SentenceTransformer('all-MiniLM-L6-v2')

        print("모든 모델 초기화 완료!")
        return llm, model, preprocess, tokenizer, nlp, embedder
    except Exception as e:
        print(f"모델 초기화 중 오류 발생: {str(e)}")
        return None, None, None, None, None, None

# 전역 변수로 모델 초기화
print("모델 초기화 시작...")
GLOBAL_LLM, GLOBAL_CLIP_MODEL, GLOBAL_CLIP_PREPROCESS, GLOBAL_CLIP_TOKENIZER, GLOBAL_NLP, GLOBAL_EMBEDDER = init_models()
print("사용 가능한 모델:", "LLaVA" if GLOBAL_LLM else "None")

모델 초기화 시작...
LLaVA 모델 로딩 중...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

WARNING 12-01 05:15:03 config.py:1865] Casting torch.bfloat16 to torch.float16.
INFO 12-01 05:15:14 config.py:350] This model supports multiple tasks: {'generate', 'embedding'}. Defaulting to 'generate'.
INFO 12-01 05:15:14 llm_engine.py:249] Initializing an LLM engine (v0.6.4.post1) with config: model='llava-hf/llava-v1.6-mistral-7b-hf', speculative_config=None, tokenizer='llava-hf/llava-v1.6-mistral-7b-hf', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=F

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

모든 모델 초기화 완료!
사용 가능한 모델: LLaVA


In [10]:
import gradio as gr
import base64
import os  # 파일 경로 관리
from PIL import Image  # 이미지 처리 라이브러리
import io  # 파일 입출력 핸들링
import torch
import faiss
import numpy as np
from vllm.sampling_params import SamplingParams

# 세션별 FAISS 인덱스 및 청크를 관리하는 클래스
class SessionIndexManager:
    def __init__(self, embedder):
        """
        초기화 메서드: 세션별 인덱스와 청크를 저장할 딕셔너리를 생성
        """
        self.embedder = embedder  # 텍스트 임베딩 생성기
        self.session_indices = {}  # 세션별 FAISS 인덱스 저장
        self.session_chunks = {}   # 세션별 문서 청크 저장

    def create_session_index(self, session_id):
        """
        새로운 세션 ID를 위한 FAISS 인덱스를 생성
        """
        dimension = self.embedder.get_sentence_embedding_dimension()  # 임베딩 차원 가져오기
        index = faiss.IndexFlatL2(dimension)  # L2 거리 기반의 평평한 FAISS 인덱스 생성
        self.session_indices[session_id] = index  # 생성된 인덱스를 세션 딕셔너리에 저장
        self.session_chunks[session_id] = []  # 해당 세션의 청크 리스트 초기화

    def add_document(self, session_id, chunks):
        """
        세션에 문서 청크를 추가하고 FAISS 인덱스에 추가
        """
        if session_id not in self.session_indices:  # 세션 인덱스가 없으면 생성
            self.create_session_index(session_id)

        embeddings = self.embedder.encode(chunks)  # 청크를 임베딩 벡터로 변환
        self.session_indices[session_id].add(np.array(embeddings))  # FAISS 인덱스에 벡터 추가
        self.session_chunks[session_id].extend(chunks)  # 청크 리스트에도 저장

    def search(self, session_id, query, k=5):
        """
        세션 인덱스를 사용해 쿼리와 유사한 상위 k개의 청크 검색
        """
        if session_id not in self.session_indices:  # 세션이 존재하지 않으면 빈 리스트 반환
            return []

        query_embedding = self.embedder.encode([query])  # 쿼리를 임베딩 벡터로 변환
        D, I = self.session_indices[session_id].search(
            np.array(query_embedding), k  # 상위 k개의 유사 벡터 검색
        )
        # 검색된 인덱스를 기반으로 세션 청크를 반환
        return [self.session_chunks[session_id][i] for i in I[0]]

    def delete_session(self, session_id):
        """
        특정 세션의 FAISS 인덱스와 청크 삭제
        """
        if session_id in self.session_indices:
            del self.session_indices[session_id]  # 세션 인덱스 제거
        if session_id in self.session_chunks:
            del self.session_chunks[session_id]  # 세션 청크 제거

# 전역 인덱스 매니저 생성
index_manager = SessionIndexManager(GLOBAL_EMBEDDER)

def detect_image_type(image):
    """
    이미지 타입을 감지하여 도메인에 맞는 분류를 반환
    """
    try:
        # CLIP 모델을 사용해 이미지 특징 추출
        image_input = GLOBAL_CLIP_PREPROCESS(image).unsqueeze(0)  # 전처리된 이미지
        categories = [  # 분류할 카테고리
            "a photo of a car or vehicle",
            "a photo of nature or landscape",
            "a financial document or spreadsheet",
            "a medical pill or medication",
            "a photo of an animal",
            "a general photograph"
        ]
        text = GLOBAL_CLIP_TOKENIZER(categories)  # 카테고리를 토큰화
        with torch.no_grad():  # 추론 시 그래디언트 비활성화
            image_features = GLOBAL_CLIP_MODEL.encode_image(image_input)  # 이미지 임베딩 생성
            text_features = GLOBAL_CLIP_MODEL.encode_text(text)  # 텍스트 임베딩 생성

            # 유사도 계산 및 정규화
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)

            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
            values, indices = similarity[0].topk(1)  # 가장 높은 유사도의 인덱스 반환

        # 카테고리 매핑
        category_mapping = {
            0: "car",
            1: "nature",
            2: "financial",
            3: "pill",
            4: "animal",
            5: "general"
        }

        detected_type = category_mapping[indices[0].item()]  # 감지된 타입
        confidence = values[0].item()  # 유사도 점수
        return detected_type

    except Exception as e:  # 예외 처리
        print(f"이미지 타입 감지 중 오류 발생: {str(e)}")
        return "general"  # 기본 타입 반환

def get_domain_specific_prompt(image_type, is_korean=True):
    """도메인별 특화된 프롬프트 생성"""
    prompts = {
        "car": {
            "ko": """자동차 이미지를 전문적으로 분석해주세요:
1. 차량 기본 정보
   - 차종 (세단, SUV, 트럭 등)
   - 외관 색상 및 디자인 특징
   - 특징적인 부분 (휠, 그릴 등)

2. 상태 및 환경
   - 차량의 전반적인 상태
   - 촬영된 환경과 각도
   - 특이사항이나 변형 여부""",

            "en": """Please analyze this vehicle image professionally:
1. Basic Vehicle Information
   - Vehicle type (sedan, SUV, truck, etc.)
   - Exterior color and design features
   - Notable parts (wheels, grille, etc.)

2. Condition and Environment
   - Overall vehicle condition
   - Photography environment and angle
   - Any modifications or special features"""
        },

        "nature": {
            "ko": """자연/풍경 이미지를 상세히 분석해주세요:
1. 주요 자연 요소
   - 지형지물 (산, 강, 바다 등)
   - 식물상 (나무, 꽃, 풀 등)
   - 기후/날씨 상태

2. 전체적인 구도
   - 원경/중경/근경의 구성
   - 빛과 그림자의 특징
   - 계절감과 시간대""",

            "en": """Please analyze this nature/landscape image in detail:
1. Key Natural Elements
   - Terrain features (mountains, rivers, ocean, etc.)
   - Flora (trees, flowers, grass, etc.)
   - Climate/weather conditions

2. Overall Composition
   - Background/middle-ground/foreground
   - Light and shadow characteristics
   - Seasonal and temporal aspects"""
        },

        "animal": {
            "ko": """동물 이미지를 매우 상세하고 정확하게 분석해주세요:

1. 종 식별 및 기본 정보
  - 동물의 정확한 종과 품종 (예: 골든 리트리버, 시베리안 허스키 등)
  - 추정 나이대 (새끼, 성견, 노견 등)
  - 개체 수와 성별 특징 (알 수 있는 경우)

2. 외모와 신체적 특징
  - 털색과 피모 상태 (구체적인 색조와 패턴)
  - 체형과 체격 (크기, 골격, 체형 등)
  - 얼굴 특징 (눈 색상, 귀 모양, 코와 입 특징)
  - 특별한 신체적 특징이나 무늬

3. 표정과 감정 상태
  - 각 개체의 구체적인 표정
  - 감정 상태나 기분 (행복해 보임, 졸려 보임 등)
  - 에너지 레벨과 활동성

4. 행동과 상호작용
  - 현재의 자세나 포즈
  - 다른 개체들과의 관계나 상호작용
  - 카메라나 주변 환경과의 반응

5. 환경과 상황
  - 촬영된 장소와 배경
  - 동물의 컨디션과 건강 상태
  - 특별한 상황이나 맥락

주의: 관찰 가능한 특징만을 바탕으로 객관적이고 구체적으로 설명해주세요.
추측이나 불확실한 정보는 제외하고 설명해주세요.""",

            "en": """Please analyze this animal image in great detail:

1. Species Identification & Basic Info
  - Exact species and breed (e.g., Golden Retriever, Siberian Husky)
  - Estimated age group (puppy, adult, senior)
  - Number of individuals and gender characteristics (if apparent)

2. Appearance & Physical Features
  - Fur color and coat condition (specific tones and patterns)
  - Body type and build (size, frame, body shape)
  - Facial features (eye color, ear shape, nose and mouth characteristics)
  - Distinctive physical marks or patterns

3. Expression & Emotional State
  - Specific facial expressions of each individual
  - Emotional state or mood (appears happy, sleepy, etc.)
  - Energy level and activity

4. Behavior & Interaction
  - Current posture or pose
  - Relationship and interaction with other individuals
  - Response to camera or surroundings

5. Environment & Context
  - Location and background of the photo
  - Animal's condition and health appearance
  - Special circumstances or context

Note: Please provide objective and specific descriptions based only on
observable characteristics. Exclude any speculation or uncertain information."""
        },

        "financial": {
            "ko": """재무 문서를 체계적으로 분석해주세요:
1. 문서 유형 확인
   - 재무제표 종류 (재무상태표, 손익계산서 등)
   - 기간 및 날짜 정보
   - 회사/기관명

2. 시각적 구성 요소
   - 표의 구조와 레이아웃
   - 주요 섹션과 카테고리
   - 데이터 표시 형식

주의: 구체적인 숫자나 민감한 재무 정보는 언급하지 않으며,
문서의 구조와 형식적 특징만을 설명합니다.""",

            "en": """Please analyze this financial document systematically:
1. Document Identification
   - Type of financial statement
   - Period and date information
   - Company/Institution name

2. Visual Components
   - Table structure and layout
   - Major sections and categories
   - Data presentation format

Note: Avoid mentioning specific numbers or sensitive financial data,
focus on document structure and formatting characteristics."""
        },

        "pill": {
            "ko": """약품 이미지를 전문적으로 분석해주세요:
1. 약품의 물리적 특성
   - 크기, 모양, 색상
   - 각인 또는 표시된 문자/숫자
   - 특별한 디자인 요소

2. 식별 가능한 정보
   - 약품의 종류나 유형
   - 제조사 표시나 브랜드 요소
   - 단위나 용량 표시

주의: 구체적인 의약품 식별이나 의학적 조언은 하지 않으며,
관찰 가능한 시각적 특징만을 객관적으로 설명해주세요.""",

            "en": """Please analyze this pharmaceutical image professionally:
1. Physical characteristics
   - Size, shape, color
   - Imprints or markings
   - Special design elements

2. Identifiable information
   - Type of medication
   - Manufacturer marks or branding
   - Unit or dosage indicators

Note: Avoid specific drug identification or medical advice,
focus only on observable visual characteristics."""
        }
    }
    return prompts.get(image_type, get_general_prompt(is_korean))

def get_general_prompt(is_korean):
    """일반적인 이미지 분석을 위한 프롬프트 생성"""
    if is_korean:
        return """이미지를 상세히 분석하여 한국어로 설명해주세요:

[사진 유형에 따른 주요 분석 요소]
1. 주요 구성 요소
   - 대상의 종류와 특징
   - 위치와 배치
   - 특징적인 모습이나 상태

2. 장면 구성
   - 전체적인 구도와 분위기
   - 배경과 주변 환경
   - 조명과 색감

3. 세부 사항
   - 특별히 눈에 띄는 요소
   - 독특한 특징이나 패턴
   - 전반적인 상태나 품질

명확하게 보이는 내용만 자연스럽게 통합하여 설명해주세요."""
    else:
        return """Please provide a detailed analysis of the image in English:

[Key Analysis Points]
1. Main Components
   - Type and characteristics of subjects
   - Position and arrangement
   - Distinctive features or conditions

2. Scene Composition
   - Overall layout and atmosphere
   - Background and surroundings
   - Lighting and color scheme

3. Details
   - Notable elements
   - Unique features or patterns
   - Overall condition or quality

Please integrate only clearly visible elements into a natural description."""

def detect_language(text):
    """
    입력된 텍스트에 한글 문자가 포함되어 있는지 확인.
    한글 유니코드 범위를 기반으로 판별.
    """
    return any('\u3131' <= char <= '\u318F' or '\uAC00' <= char <= '\uD7A3' for char in text)

def process_message(message, file, mode, history, session_name):
    """
    사용자의 메시지 및 업로드된 파일을 처리하여 응답 생성.
    General Chat과 RAG Chat 두 가지 모드를 지원.
    """
    is_korean = detect_language(message)  # 텍스트가 한글인지 여부 확인

    def post_process_response(response):
        """
        응답의 문장을 고유하게 처리하여 중복 제거.
        """
        sentences = response.split('.')  # 응답을 문장 단위로 분리
        unique_sentences = []  # 고유 문장을 저장할 리스트
        for s in sentences:
            s = s.strip()  # 앞뒤 공백 제거
            if s and s not in unique_sentences:  # 중복되지 않은 경우 추가
                unique_sentences.append(s)
        return '. '.join(unique_sentences)  # 고유 문장들만 결합하여 반환

    if mode == "General Chat":  # 일반 채팅 모드
        if not file:  # 파일이 없을 경우 텍스트 처리
            try:
                if history and history[-1][1]:  # 이전 대화 내역이 있는 경우
                    if is_korean:
                        prompt = f"""이전 대화 내용: {history[-1][1]}
새로운 질문: {message}
이전 대화 내용을 참고하여 새로운 질문에 답변해주세요."""
                    else:
                        prompt = f"""Previous context: {history[-1][1]}
New question: {message}
Please answer the new question considering the previous context."""
                else:
                    prompt = message  # 대화 내역이 없으면 메시지만 사용

                # 샘플링 파라미터 설정
                sampling_params = SamplingParams(
                    temperature=0.15,
                    max_tokens=512,
                    top_p=0.85,
                    repetition_penalty=1.18
                )

                # GPT 모델 호출하여 응답 생성
                response = GLOBAL_LLM.generate(
                    prompts=[prompt],
                    sampling_params=sampling_params
                )
                return post_process_response(response[0].outputs[0].text.strip())
            except Exception as e:  # 예외 처리
                return f"텍스트 처리 중 오류 발생: {str(e)}" if is_korean else f"Error processing text: {str(e)}"
        else:  # 파일이 있는 경우 이미지 처리
            try:
                image = Image.open(file)  # 파일에서 이미지 로드
                image_type = detect_image_type(image)  # 이미지 타입 감지
                base_prompt = get_domain_specific_prompt(image_type, is_korean)  # 도메인에 따른 프롬프트 생성

                question = message if message.strip() else "이 이미지를 설명해주세요."  # 메시지가 없으면 기본 질문 사용
                prompt = f"""[INST] <image>
이미지를 보고 설명해주세요:

{base_prompt}

질문: {question}

주의사항:
- 관찰된 내용만 설명
- 한국어로 자연스럽게 작성 [/INST]"""

                # 입력 데이터 설정
                inputs = {
                    "prompt": prompt,
                    "multi_modal_data": {"image": image},
                }

                # 샘플링 파라미터 설정
                sampling_params = SamplingParams(
                    temperature=0.15,
                    max_tokens=512,
                    top_p=0.85,
                    repetition_penalty=1.18
                )

                try:
                    # GPT 모델 호출하여 이미지 설명 생성
                    outputs = GLOBAL_LLM.generate(inputs, sampling_params=sampling_params)
                    if outputs and len(outputs) > 0:
                        response = outputs[0].outputs[0].text.strip()  # 응답 텍스트 추출
                        if not response:
                            return "응답을 생성하지 못했습니다." if is_korean else "Failed to generate response."
                        return post_process_response(response)  # 응답 후처리 및 반환
                finally:
                    # 메모리 관리: 불필요한 객체 삭제 및 캐시 정리
                    del inputs
                    if 'outputs' in locals():
                        del outputs
                    import gc
                    gc.collect()
                    torch.cuda.empty_cache()

            except Exception as e:  # 예외 처리
                print(f"디버그 - 에러 상세: {str(e)}")
                return f"이미지 처리 중 오류 발생: {str(e)}" if is_korean else f"Error processing image: {str(e)}"

    else:  # RAG Chat 모드
        try:
            if file and file.name.endswith('.pdf'):  # 업로드된 파일이 PDF인 경우
                chunks = process_pdf_to_chunks(file)  # PDF 파일을 청크로 처리 (별도 구현 필요)
                index_manager.add_document(session_name, chunks)  # 세션 인덱스에 문서 추가
                return "PDF가 성공적으로 처리되었습니다." if is_korean else "PDF processed successfully."

            # 세션에서 쿼리와 관련된 청크 검색
            relevant_chunks = index_manager.search(session_name, message)
            if not relevant_chunks:  # 관련 청크가 없을 경우
                return "관련 정보를 찾을 수 없습니다." if is_korean else "No relevant information found."

            # 검색된 청크를 기반으로 프롬프트 생성
            context = "\n".join(relevant_chunks)
            if is_korean:
                prompt = f"""다음 컨텍스트를 바탕으로 질문에 답변해주세요:

컨텍스트:
{context}

질문: {message}

주의사항:
- 주어진 컨텍스트 내용만 사용하여 답변
- 확실하지 않은 정보는 언급하지 않음
- 컨텍스트에 없는 내용은 모른다고 답변"""
            else:
                prompt = f"""Please answer the question based on the following context:

Context:
{context}

Question: {message}

Note:
- Use only the information from the given context
- Don't mention uncertain information
- If information is not in the context, acknowledge that you don't know"""
## 여기까지 RAG

            # 샘플링 파라미터 설정
            sampling_params = SamplingParams(
                temperature=0.15,
                max_tokens=512,
                top_p=0.85,
                repetition_penalty=1.18
            )

            # GPT 모델 호출하여 응답 생성
            response = GLOBAL_LLM.generate(
                prompts=[prompt],
                sampling_params=sampling_params
            )
            return post_process_response(response[0].outputs[0].text.strip())

        except Exception as e:  # 예외 처리
            return f"RAG 처리 중 오류 발생: {str(e)}" if is_korean else f"Error in RAG processing: {str(e)}"

def create_ui():
    """
    Gradio UI를 생성하고 이벤트를 연결하여 사용자와 상호작용.
    사용자 상태(state) 관리 및 다중 세션 지원.
    """
    custom_css = """
    .message-box {
        display: flex;
        align-items: center;
        gap: 0.5rem;
    }
    .file-btn {
        max-width: 40px;
    }
    .send-btn {
        max-width: 40px;
    }
    .selected-file {
        margin: 0.5rem 0;
        padding: 0.3rem;
        background: #f0f0f0;
        border-radius: 4px;
        font-size: 0.9em;
    }
    """
    # 사용자 상태 초기화
    state = {
        "sessions": {  # 세션별 히스토리와 모드를 관리
            "Example": {"history": [], "mode": "General Chat"}
        },
        "current_session": "Example"  # 기본 세션 설정
    }

    with gr.Blocks(css=custom_css) as demo:  # Gradio Blocks로 UI 생성
        with gr.Row():  # 상단에 세션 패널과 채팅 패널을 나란히 배치
            # 왼쪽 세션 패널
            with gr.Column(scale=1):  # 작은 비율(1)로 세션 관리 패널 배치
                new_session_btn = gr.Button("+ New Session")  # 새로운 세션 생성 버튼
                session_title_input = gr.Textbox(  # 새 세션 이름 입력란
                    label="Session Title",
                    visible=False  # 기본적으로 숨김
                )
                with gr.Column(elem_classes="session-container"):  # 세션 리스트 표시
                    gr.Markdown("Sessions")  # 세션 제목
                    session_list = gr.Radio(  # 세션 선택 라디오 버튼
                        choices=["Example"],  # 기본 세션 포함
                        value="Example",  # 기본 선택된 세션
                        interactive=True,  # 사용자 상호작용 가능
                        label=""  # 라벨 제거
                    )
                    delete_btn = gr.Button("🗑️ Delete Selected Session")  # 세션 삭제 버튼

            # 오른쪽 채팅 패널
            with gr.Column(scale=3):  # 큰 비율(3)로 채팅 패널 배치
                current_title = gr.Markdown("## Example")  # 현재 세션 제목 표시

                with gr.Row():  # 채팅 모드 선택 라디오 버튼
                    chat_mode = gr.Radio(
                        choices=["General Chat", "RAG Chat"],  # 지원되는 두 가지 모드
                        value="General Chat",  # 기본 모드는 General Chat
                        interactive=True,  # 사용자 상호작용 가능
                        label=""
                    )

                with gr.Column():  # 채팅 인터페이스
                    chatbot = gr.Chatbot(  # 채팅 기록 창
                        height=400,  # 높이 설정
                        render_markdown=True,  # 마크다운 렌더링 지원
                        show_copy_button=True,  # 응답 복사 버튼 표시
                        bubble_full_width=False  # 메시지 말풍선 크기 고정
                    )

                    with gr.Row():  # 메시지 입력 및 파일 업로드 행
                        file_upload = gr.UploadButton(  # 파일 업로드 버튼
                            "📎",
                            file_types=["image", "pdf"],  # 지원되는 파일 형식
                            scale=1  # 버튼 크기
                        )
                        msg = gr.Textbox(  # 메시지 입력 텍스트박스
                            show_label=False,  # 라벨 숨김
                            placeholder="Type your message...",  # 플레이스홀더
                            container=False,  # 컨테이너 비활성화
                            scale=44  # 입력란 크기 비율
                        )
                        send_btn = gr.Button(  # 메시지 전송 버튼
                            "↑",
                            scale=1  # 버튼 크기 비율
                        )

                    with gr.Row():  # 추가 옵션 버튼 행
                        clear_btn = gr.Button("Clear Chat")  # 채팅 기록 초기화 버튼
                    with gr.Row():
                        selected_file = gr.Textbox(  # 업로드된 파일 이름 표시
                            label="Selected File",
                            interactive=False  # 수정 불가
                        )

        # ----------- 이벤트 핸들러 정의 -----------

        def send_message(message, file, session_name, mode, history):
            """
            사용자가 입력한 메시지와 업로드한 파일을 처리하여 응답 생성.
            """
            if not message.strip() and not file:  # 메시지와 파일이 모두 없으면 처리하지 않음
                return history, "", None, None

            try:
                if file:  # 파일 업로드가 있을 경우
                    user_message = f"{message}"  # 메시지 저장
                    history.append(((file.name, file), user_message))  # 히스토리에 저장
                    response = process_message(message, file, mode, history, session_name)  # 메시지 처리
                    history.append((None, response))  # 응답 추가
                else:  # 파일이 없을 경우 텍스트만 처리
                    history.append((None, message))  # 메시지 추가
                    response = process_message(message, file, mode, history, session_name)  # 메시지 처리
                    history.append((None, response))  # 응답 추가

                # 현재 세션의 히스토리 업데이트
                if session_name in state["sessions"]:
                    state["sessions"][session_name]["history"] = history

                return history, "", None, None  # 새로운 히스토리와 UI 초기화 반환
            except Exception as e:  # 예외 처리
                print(f"메시지 전송 중 오류 발생: {str(e)}")
                return history, "", None, None

        def show_title_input():
            """
            새로운 세션 제목 입력란을 표시.
            """
            return gr.update(visible=True)

        def add_session(title):
            """
            새로운 세션 추가. 이름이 중복되거나 비어있으면 추가하지 않음.
            """
            if not title or title in state["sessions"]:  # 이름이 유효하지 않거나 중복된 경우
                return (
                    gr.update(visible=False),  # 입력란 숨김
                    gr.update(choices=list(state["sessions"].keys()), value=state["current_session"])  # 기존 세션 리스트 반환
                )

            # 새로운 세션 추가
            new_sessions = {title: {"history": [], "mode": "General Chat"}}
            new_sessions.update(state["sessions"])  # 기존 세션에 추가
            state["sessions"] = new_sessions
            state["current_session"] = title  # 현재 세션 업데이트

            choices = list(state["sessions"].keys())  # 세션 목록 생성

            return (
                gr.update(visible=False),  # 입력란 숨김
                gr.update(choices=choices, value=title)  # 세션 목록과 새 선택값 반환
            )

        def switch_session(session_name):
            """
            선택된 세션으로 전환하고 히스토리와 모드 동기화.
            """
            if session_name in state["sessions"]:  # 선택된 세션이 존재하면
                state["current_session"] = session_name  # 현재 세션 업데이트
                return (
                    f"## {session_name}",  # 제목 업데이트
                    state["sessions"][session_name]["history"],  # 해당 세션 히스토리
                    state["sessions"][session_name]["mode"]  # 세션 모드
                )
            return current_title, [], chat_mode.value  # 기본값 반환

        def delete_session(session_name):
            """
            선택된 세션 삭제. 마지막 세션은 삭제 불가.
            """
            if len(state["sessions"]) <= 1:  # 최소 1개의 세션은 유지해야 함
                gr.Warning("마지막 세션은 삭제할 수 없습니다.")
                return (
                    gr.update(choices=list(state["sessions"].keys()), value=session_name),
                    current_title,
                    chatbot,
                    chat_mode
                )

            if session_name in state["sessions"]:  # 선택된 세션 삭제
                del state["sessions"][session_name]  # 상태에서 세션 삭제
                index_manager.delete_session(session_name)  # 세션 인덱스 삭제
                next_session = next(iter(state["sessions"].keys()))  # 다음 세션으로 이동
                state["current_session"] = next_session
                return (
                    gr.update(choices=list(state["sessions"].keys()), value=next_session),
                    f"## {next_session}",
                    state["sessions"][next_session]["history"],
                    state["sessions"][next_session]["mode"]
                )
            return (
                gr.update(choices=list(state["sessions"].keys())),
                current_title,
                chatbot,
                chat_mode
            )

        def chat_mode_change(mode):
            """
            선택된 채팅 모드에 따라 파일 업로드 허용 형식을 변경.
            """
            session_name = state["current_session"]
            if session_name in state["sessions"]:  # 현재 세션 모드 업데이트
                state["sessions"][session_name]["mode"] = mode
            allowed_types = [".jpg", ".jpeg", ".png"] if mode == "General Chat" else [".pdf"]
            return gr.update(file_types=allowed_types)  # 허용 파일 유형 반환

        def update_selected_file(file):
            """
            업로드된 파일 이름을 반환.
            """
            return file.name if file else ""

        def clear_chat():
            """
            현재 세션의 채팅 기록을 초기화.
            """
            session_name = state["current_session"]
            if session_name in state["sessions"]:
                state["sessions"][session_name]["history"] = []  # 히스토리 초기화
                index_manager.delete_session(session_name)  # 세션 인덱스 초기화
            return [], None  # 초기화된 UI 반환

        # ----------- Gradio 이벤트 연결 -----------

        new_session_btn.click(
            show_title_input,  # 새로운 세션 추가 버튼 클릭 시 입력란 표시
            outputs=session_title_input
        )

        session_title_input.submit(
            add_session,  # 세션 제목 입력 후 제출 시 새로운 세션 추가
            inputs=[session_title_input],
            outputs=[session_title_input, session_list]
        )

        session_list.change(
            switch_session,  # 세션 선택 변경 시 해당 세션으로 전환
            inputs=[session_list],
            outputs=[current_title, chatbot, chat_mode]
        )

        delete_btn.click(
            delete_session,  # 세션 삭제 버튼 클릭 시 세션 삭제
            inputs=[session_list],
            outputs=[session_list, current_title, chatbot, chat_mode]
        )

        chat_mode.change(
            chat_mode_change,  # 채팅 모드 변경 시 파일 업로드 형식 변경
            inputs=[chat_mode],
            outputs=[file_upload]
        )

        file_upload.upload(
            fn=update_selected_file,  # 파일 업로드 시 파일 이름 반환
            inputs=[file_upload],
            outputs=[selected_file]
        )

        send_btn.click(
            send_message,  # 메시지 전송 버튼 클릭 시 메시지 처리
            inputs=[msg, file_upload, session_list, chat_mode, chatbot],
            outputs=[chatbot, msg, file_upload, selected_file]
        )

        msg.submit(
            send_message,  # 메시지 입력 후 Enter 키로 메시지 처리
            inputs=[msg, file_upload, session_list, chat_mode, chatbot],
            outputs=[chatbot, msg, file_upload, selected_file]
        )

        clear_btn.click(
            clear_chat,  # 채팅 초기화 버튼 클릭 시 기록 삭제
            outputs=[chatbot, selected_file]
        )

        return demo  # Gradio 인터페이스 반환

if __name__ == "__main__":
    demo = create_ui()  # UI 생성
    demo.launch()  # 웹 애플리케이션 실행


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://197d66077afdebcede.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
